# Milestone 2 

In [1]:
import pandas as pd
import numpy as np

In [2]:
!kaggle datasets download -d rohanrao/formula-1-world-championship-1950-2020
!chmod 600 /Users/lewistu/.kaggle/kaggle.json

Dataset URL: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
License(s): CC0-1.0
formula-1-world-championship-1950-2020.zip: Skipping, found more recently modified local copy (use --force to force download)


Replace '\N' strings with NaNs to represent missing data.

Core datasets:
- Lap times
- Pit stops
- Races
- Results
- Status
- Circuits

Nice-to-have datasets:
- Drivers
- Constructors

In [3]:
# Replace this with your file path
data_path = '/Users/lewistu/Documents/LewisTu/Harvard College/F25 Classes/CS 109A/cs109a_project/formula-1-world-championship-1950-2020/'

try:
    circuits = pd.read_csv(data_path + 'circuits.csv', na_values=r'\N')
    lap_times = pd.read_csv(data_path + 'lap_times.csv', na_values=r'\N')
    pit_stops = pd.read_csv(data_path + 'pit_stops.csv', na_values=r'\N')
    races = pd.read_csv(data_path + 'races.csv', na_values=r'\N')
    results = pd.read_csv(data_path + 'results.csv', na_values=r'\N')
    status = pd.read_csv(data_path + 'status.csv', na_values=r'\N')

    drivers      = pd.read_csv(data_path + 'drivers.csv', na_values=r'\N')
    constructors = pd.read_csv(data_path + 'constructors.csv', na_values=r'\N')
    
    print("Files loaded successfully!")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")

Files loaded successfully!


## Filter for Hybrid era

Current era of F1 is the Hybrid (V8) era, which started in the 2014 season. This is the latest major regulation overhaul in F1, involving completely new power units for cars, which directly affects how cars manage energy, tire wear, and cooling - which affects pit stop timing. Thus we want to filter for 2014 and after.

In [4]:
MIN_YEAR = 2014

modern_races = races[races['year'] >= MIN_YEAR]
modern_race_ids = set(modern_races['raceId'])

## Building an undercut attempts dataset

In [5]:
WINDOW_LAPS = 5 # undercut window: rival must pit within this many laps

lt = lap_times[lap_times['raceId'].isin(modern_race_ids)].copy()
ps = pit_stops[pit_stops['raceId'].isin(modern_race_ids)].copy()
res = results[results['raceId'].isin(modern_race_ids)].copy()

In [6]:
lt = lt.sort_values(['raceId', 'driverId', 'lap'])
lt['cum_ms'] = lt.groupby(['raceId', 'driverId'])['milliseconds'].cumsum()

# Recent pace = mean of previous 3 laps
lt['prev3_mean_ms'] = lt.groupby(['raceId','driverId'])['milliseconds'] \
                        .transform(lambda s: s.shift(1).rolling(3, min_periods=1).mean())

In [7]:
lt.head()

,raceId,driverId,lap,position,time,milliseconds,cum_ms,prev3_mean_ms
72962,900,1,1,4,1:46.128,106128,106128,NaN
72963,900,1,2,5,1:40.287,100287,206415,106128.0
72130,900,3,1,1,1:42.038,102038,102038,NaN
72131,900,3,2,1,1:37.687,97687,199725,102038.0
72132,900,3,3,1,1:35.765,95765,295490,99862.5


In [8]:
# Identify the car immediately ahead and gap at end of each lap
lap_order = lt.copy()

ahead_map = lap_order[['raceId','lap','position','driverId','cum_ms']].copy()
ahead_map['position'] = ahead_map['position'] + 1  
ahead_map = ahead_map.rename(columns={'driverId':'ahead_driverId','cum_ms':'ahead_cum_ms'})

lap_order = lap_order.merge(ahead_map, on=['raceId','lap','position'], how='left')
lap_order['gap_to_ahead_ms'] = lap_order['cum_ms'] - lap_order['ahead_cum_ms']  # positive if behind

In [9]:
lap_order.head()

,raceId,driverId,lap,position,time,milliseconds,cum_ms,prev3_mean_ms,ahead_driverId,ahead_cum_ms,gap_to_ahead_ms
0,900,1,1,4,1:46.128,106128,106128,NaN,825.0,105402.0,726.0
1,900,1,2,5,1:40.287,100287,206415,106128.0,807.0,205345.0,1070.0
2,900,3,1,1,1:42.038,102038,102038,NaN,NaN,NaN,NaN
3,900,3,2,1,1:37.687,97687,199725,102038.0,NaN,NaN,NaN
4,900,3,3,1,1:35.765,95765,295490,99862.5,NaN,NaN,NaN


In [10]:
# Merge pit info per lap
pits_one_per_lap = (ps.sort_values(['raceId','driverId','lap','stop'])
                      .drop_duplicates(['raceId','driverId','lap'], keep='first')
                      [['raceId','driverId','lap','milliseconds']]
                      .rename(columns={'milliseconds':'pit_ms'}))

lap_order = lap_order.merge(
    pits_one_per_lap.assign(pit_flag=1),
    on=['raceId','driverId','lap'], how='left'
)
lap_order['pit_flag'] = lap_order['pit_flag'].fillna(0).astype(int)

In [11]:
# Stint info (laps since last pit and stint number)
tmp = lap_order[['raceId','driverId','lap','pit_flag']].copy()
tmp['last_pit_lap'] = np.where(tmp['pit_flag'].eq(1), tmp['lap'], np.nan)
tmp['last_pit_lap'] = tmp.sort_values(['raceId','driverId','lap']) \
                       .groupby(['raceId','driverId'])['last_pit_lap'] \
                       .ffill().fillna(0)

lap_order['laps_since_last_pit'] = lap_order['lap'] - tmp['last_pit_lap']
lap_order['stint_no'] = (lap_order.sort_values(['raceId','driverId','lap'])
                         .groupby(['raceId','driverId'])['pit_flag'].cumsum().astype(int) + 1)

In [12]:
lap_order.head()

,raceId,driverId,lap,position,time,milliseconds,cum_ms,prev3_mean_ms,ahead_driverId,ahead_cum_ms,gap_to_ahead_ms,pit_ms,pit_flag,laps_since_last_pit,stint_no
0,900,1,1,4,1:46.128,106128,106128,NaN,825.0,105402.0,726.0,NaN,0,1.0,1
1,900,1,2,5,1:40.287,100287,206415,106128.0,807.0,205345.0,1070.0,NaN,0,2.0,1
2,900,3,1,1,1:42.038,102038,102038,NaN,NaN,NaN,NaN,NaN,0,1.0,1
3,900,3,2,1,1:37.687,97687,199725,102038.0,NaN,NaN,NaN,NaN,0,2.0,1
4,900,3,3,1,1:35.765,95765,295490,99862.5,NaN,NaN,NaN,NaN,0,3.0,1


In [13]:
# Pit events for the trailing car (B)
prev_state = lap_order[['raceId','driverId','lap','position','ahead_driverId','gap_to_ahead_ms',
                        'prev3_mean_ms','laps_since_last_pit','stint_no']].copy()
prev_state = prev_state.rename(columns={
    'position':'b_position_prev',
    'ahead_driverId':'a_driverId',
    'gap_to_ahead_ms':'gap_prev_ms',
    'prev3_mean_ms':'b_prev3_mean_ms',
    'laps_since_last_pit':'b_laps_since_last_pit',
    'stint_no':'b_stint_no'
})
prev_state['lap'] = prev_state['lap'] + 1  # so we can join to pit lap

pit_events = lap_order[lap_order['pit_flag'].eq(1)][['raceId','driverId','lap','pit_ms']].copy()
pit_events = pit_events.merge(prev_state, on=['raceId','driverId','lap'], how='left')
pit_events = pit_events.rename(columns={'driverId':'b_driverId'})

# Drop events without a car ahead
pit_events = pit_events[~pit_events['a_driverId'].isna()].copy()

In [14]:
# Add pre-pit metrics taken at lap-1 for the car ahead (A)
a_prev_metrics = lap_order[['raceId','driverId','lap','prev3_mean_ms','laps_since_last_pit','stint_no']].copy()
a_prev_metrics = a_prev_metrics.rename(columns={
    'driverId':'a_driverId',
    'prev3_mean_ms':'a_prev3_mean_ms',
    'laps_since_last_pit':'a_laps_since_last_pit',
    'stint_no':'a_stint_no'
})
a_prev_metrics['lap'] = a_prev_metrics['lap'] + 1
pit_events = pit_events.merge(a_prev_metrics, on=['raceId','a_driverId','lap'], how='left')

In [15]:
pit_events.head()

,raceId,b_driverId,lap,pit_ms,b_position_prev,a_driverId,gap_prev_ms,b_prev3_mean_ms,b_laps_since_last_pit,b_stint_no,a_prev3_mean_ms,a_laps_since_last_pit,a_stint_no
0,900,4,12,22887.0,5.0,807.0,1387.0,96619.000000,11.0,1.0,96561.666667,11.0,1
1,900,4,35,21978.0,4.0,825.0,7569.0,95837.666667,22.0,2.0,95443.666667,22.0,2
2,900,8,12,25543.0,6.0,4.0,2574.0,97071.333333,11.0,1.0,96619.000000,11.0,1
3,900,8,36,21825.0,5.0,822.0,1712.0,95758.333333,23.0,2.0,95059.666667,25.0,2
4,900,16,35,24305.0,11.0,818.0,4998.0,97173.000000,34.0,1.0,102226.000000,1.0,3


In [16]:
# Find A's next pit after B's pit (undercut window)
a_pits_all = ps[['raceId','driverId','lap','milliseconds']].rename(
    columns={'driverId':'a_driverId','lap':'a_pit_lap','milliseconds':'a_pit_ms'}
)
# Join and keep the first A pit strictly after B's pit (lap) and within WINDOW_LAPS
tmp = pit_events.merge(a_pits_all, on=['raceId','a_driverId'], how='left')
tmp = tmp[tmp['a_pit_lap'] > tmp['lap']]
tmp = tmp[tmp['a_pit_lap'] <= tmp['lap'] + WINDOW_LAPS]
tmp = tmp.sort_values(['raceId','b_driverId','lap','a_pit_lap'])
tmp = tmp.drop_duplicates(subset=['raceId','b_driverId','lap'], keep='first')
pit_events = tmp

In [17]:
pit_events.head()

,raceId,b_driverId,lap,pit_ms,b_position_prev,a_driverId,gap_prev_ms,b_prev3_mean_ms,b_laps_since_last_pit,b_stint_no,a_prev3_mean_ms,a_laps_since_last_pit,a_stint_no,a_pit_lap,a_pit_ms
3,900,4,35,21978.0,4.0,825.0,7569.0,95837.666667,22.0,2.0,95443.666667,22.0,2,37.0,22273.0
10,900,18,11,22411.0,9.0,826.0,1218.0,96109.666667,10.0,1.0,95927.000000,10.0,1,12.0,30514.0
13,900,18,32,22399.0,6.0,4.0,1425.0,95931.000000,20.0,2.0,95917.666667,19.0,2,35.0,21978.0
16,900,154,28,22264.0,13.0,815.0,5871.0,96863.000000,26.0,2.0,96819.000000,16.0,3,32.0,22526.0
20,900,807,33,22933.0,4.0,825.0,7148.0,95955.000000,20.0,2.0,95832.000000,20.0,2,37.0,22273.0


In [18]:
# Determine positions immediately after Driver A's pit stop
pos_at_lap = lap_order[['raceId','driverId','lap','position']].copy()

# A's position at end of its pit lap
a_post = pos_at_lap.rename(columns={'driverId':'a_driverId','position':'a_position_post','lap':'pos_lap'})
pit_events = pit_events.merge(
    a_post, left_on=['raceId','a_driverId','a_pit_lap'],
    right_on=['raceId','a_driverId','pos_lap'], how='left'
).drop(columns=['pos_lap'])

# B's position at end of A's pit lap
b_post = pos_at_lap.rename(columns={'driverId':'b_driverId','position':'b_position_post','lap':'pos_lap'})
pit_events = pit_events.merge(
    b_post, left_on=['raceId','b_driverId','a_pit_lap'],
    right_on=['raceId','b_driverId','pos_lap'], how='left'
).drop(columns=['pos_lap'])

# Label success: B ahead of A after A's pit lap
pit_events['undercut_success'] = (pit_events['b_position_post'] < pit_events['a_position_post']).astype(int)

In [19]:
pit_events.head()

,raceId,b_driverId,lap,pit_ms,b_position_prev,a_driverId,gap_prev_ms,b_prev3_mean_ms,b_laps_since_last_pit,b_stint_no,a_prev3_mean_ms,a_laps_since_last_pit,a_stint_no,a_pit_lap,a_pit_ms,a_position_post,b_position_post,undercut_success
0,900,4,35,21978.0,4.0,825.0,7569.0,95837.666667,22.0,2.0,95443.666667,22.0,2,37.0,22273.0,2,5.0,0
1,900,18,11,22411.0,9.0,826.0,1218.0,96109.666667,10.0,1.0,95927.000000,10.0,1,12.0,30514.0,9,6.0,1
2,900,18,32,22399.0,6.0,4.0,1425.0,95931.000000,20.0,2.0,95917.666667,19.0,2,35.0,21978.0,7,8.0,0
3,900,154,28,22264.0,13.0,815.0,5871.0,96863.000000,26.0,2.0,96819.000000,16.0,3,32.0,22526.0,12,13.0,0
4,900,807,33,22933.0,4.0,825.0,7148.0,95955.000000,20.0,2.0,95832.000000,20.0,2,37.0,22273.0,2,6.0,0


In [20]:
# Add race and grid data
pit_events = pit_events.merge(modern_races, on='raceId', how='left')

res_meta = res[['raceId','driverId','constructorId','grid']].copy()
pit_events = pit_events.merge(
    res_meta.rename(columns={'driverId':'b_driverId','constructorId':'b_constructorId','grid':'b_grid'}),
    on=['raceId','b_driverId'], how='left'
)
pit_events = pit_events.merge(
    res_meta.rename(columns={'driverId':'a_driverId','constructorId':'a_constructorId','grid':'a_grid'}),
    on=['raceId','a_driverId'], how='left'
)


In [21]:
pit_events.head()

,raceId,b_driverId,lap,pit_ms,b_position_prev,a_driverId,gap_prev_ms,b_prev3_mean_ms,b_laps_since_last_pit,b_stint_no,...,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,b_constructorId,b_grid,a_constructorId,a_grid
0,900,4,35,21978.0,4.0,825.0,7569.0,95837.666667,22.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6,5,1,4
1,900,18,11,22411.0,9.0,826.0,1218.0,96109.666667,10.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,10,5,8
2,900,18,32,22399.0,6.0,4.0,1425.0,95931.000000,20.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,10,6,5
3,900,154,28,22264.0,13.0,815.0,5871.0,96863.000000,26.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,208,22,10,16
4,900,807,33,22933.0,4.0,825.0,7148.0,95955.000000,20.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10,7,1,4


In [22]:
# Feature set
undercuts = pit_events[[
    'raceId','year','round','name','circuitId',
    'b_driverId','a_driverId',
    'lap',  # B pit lap 
    'a_pit_lap',
    'gap_prev_ms',  # gap to A at end of lap before B pits
    'b_prev3_mean_ms','a_prev3_mean_ms',
    'b_laps_since_last_pit','a_laps_since_last_pit',
    'b_stint_no','a_stint_no',
    'pit_ms','a_pit_ms',
    'b_position_prev','b_position_post','a_position_post',
    'b_constructorId','a_constructorId',
    'b_grid','a_grid',
    'undercut_success'
]].rename(columns={'lap':'b_pit_lap'})

undercuts['delta_prev3_ms'] = undercuts['b_prev3_mean_ms'] - undercuts['a_prev3_mean_ms']

In [23]:
# columns that should be whole numbers
int_cols = [
    'raceId','year','round','circuitId',
    'b_driverId','a_driverId',
    'b_pit_lap','a_pit_lap',
    'b_laps_since_last_pit','a_laps_since_last_pit',
    'b_stint_no','a_stint_no',
    'b_position_prev','b_position_post','a_position_post',
    'b_constructorId','a_constructorId','b_grid','a_grid',
    'undercut_success',
    'pit_ms', 'a_pit_ms'
]

undercuts[int_cols] = undercuts[int_cols].round().astype('Int64')

In [24]:
undercuts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2397 entries, 0 to 2396
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   raceId                 2397 non-null   Int64  
 1   year                   2397 non-null   Int64  
 2   round                  2397 non-null   Int64  
 3   name                   2397 non-null   object 
 4   circuitId              2397 non-null   Int64  
 5   b_driverId             2397 non-null   Int64  
 6   a_driverId             2397 non-null   Int64  
 7   b_pit_lap              2397 non-null   Int64  
 8   a_pit_lap              2397 non-null   Int64  
 9   gap_prev_ms            2397 non-null   float64
 10  b_prev3_mean_ms        2305 non-null   float64
 11  a_prev3_mean_ms        2305 non-null   float64
 12  b_laps_since_last_pit  2397 non-null   Int64  
 13  a_laps_since_last_pit  2397 non-null   Int64  
 14  b_stint_no             2397 non-null   Int64  
 15  a_st

In [25]:
print(undercuts['undercut_success'].value_counts(dropna=False))
print("Success rate:", round(undercuts['undercut_success'].mean(), 3))

undercut_success
0    2256
1     141
Name: count, dtype: Int64
Success rate: 0.059


In [26]:
undercuts.head()

,raceId,year,round,name,circuitId,b_driverId,a_driverId,b_pit_lap,a_pit_lap,gap_prev_ms,...,a_pit_ms,b_position_prev,b_position_post,a_position_post,b_constructorId,a_constructorId,b_grid,a_grid,undercut_success,delta_prev3_ms
0,900,2014,1,Australian Grand Prix,1,4,825,35,37,7569.0,...,22273,4,5,2,6,1,5,4,0,394.000000
1,900,2014,1,Australian Grand Prix,1,18,826,11,12,1218.0,...,30514,9,6,9,1,5,10,8,1,182.666667
2,900,2014,1,Australian Grand Prix,1,18,4,32,35,1425.0,...,21978,6,8,7,1,6,10,5,0,13.333333
3,900,2014,1,Australian Grand Prix,1,154,815,28,32,5871.0,...,22526,13,13,12,208,10,22,16,0,44.000000
4,900,2014,1,Australian Grand Prix,1,807,825,33,37,7148.0,...,22273,4,6,2,10,1,7,4,0,123.000000
